<a href="https://colab.research.google.com/github/toddlack/colab/blob/main/coquiTTS_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


Copy files from google drive to session's  /content folder

In [2]:
from google.colab import files
%mkdir -p /content/tts_models/tammy09apr
%cp /drive/MyDrive/ai_voice/tammy09apr-300epochs/tammy09apr-300epochs9300.pth /content/tts_models/tammy09apr

Install Dependencies

In [3]:
# @title Clone repo for local use
#Clone a repo and cd into the new directory
content_root = "/content" # @param {"type":"string","placeholder":"content root"}
git_repo='https://github.com/coqui-ai/TTS.git'  # @param {"type":"string","placeholder":"git repo url"}
git_branch='dev'  # @param {"type":"string","placeholder":"git branch"}
app_root=git_repo.split('/')[-1].replace('.git', '')
assets_dir=f'{content_root}/{app_root}/assets'
%cd {content_root}
!git clone --single-branch -b {git_branch} {git_repo}
%cd {app_root}

/content
fatal: destination path 'TTS' already exists and is not an empty directory.
/content/TTS


Copy dataset (audio) to train

In [4]:
# prompt: copy directory from dataset_source variable to content_root/dataset

# Copy dataset (audio) to train
dataset_source = '/drive/MyDrive/ai_voice/training_audio/tammy' # @param {"type":"string","placeholder":"path to source audio dataset"}
dataset_destination = 'dataset/audio' # @param {"type":"string","placeholder":"path to audio destination"}
%mkdir -p "{content_root}/{dataset_destination}"
%cp -rv "{dataset_source}" "{content_root}/{dataset_destination}"

'/drive/MyDrive/ai_voice/training_audio/tammy/aiana-105-Lesson-VO7.7-central-tendency.mp3' -> '/content/dataset/audio/tammy/aiana-105-Lesson-VO7.7-central-tendency.mp3'
'/drive/MyDrive/ai_voice/training_audio/tammy/aiana-105-Lesson-VO5.6-data-analysis-defined.mp3' -> '/content/dataset/audio/tammy/aiana-105-Lesson-VO5.6-data-analysis-defined.mp3'
'/drive/MyDrive/ai_voice/training_audio/tammy/exercise-3.4.mp3' -> '/content/dataset/audio/tammy/exercise-3.4.mp3'
'/drive/MyDrive/ai_voice/training_audio/tammy/training_tammy.mp3' -> '/content/dataset/audio/tammy/training_tammy.mp3'
'/drive/MyDrive/ai_voice/training_audio/tammy/life-underground.mp3' -> '/content/dataset/audio/tammy/life-underground.mp3'
'/drive/MyDrive/ai_voice/training_audio/tammy/ireland_script.mp3' -> '/content/dataset/audio/tammy/ireland_script.mp3'


In [6]:
# prompt: python code to generate text to voice using coqui TTS and pytorch
%pip install numpy==1.22 # Install the required NumPy version
%pip install TTS --upgrade --no-cache-dir # Reinstall/upgrade TTS after fixing NumPy, to ensure it links against the correct NumPy version
%pip install torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.22.0 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.22.0 which is incompatible.
arviz 0.20.0 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.
astropy 6.1.7 requires numpy>=1.23, but you have numpy 1.22.0 which is incompatible.
bigframes 1.29.0 requires numpy>=1.24.0, but you have numpy 1.22.0 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.22.0 which is incompatible.
contourpy 1.3.1 requires numpy>=1.23, but you have numpy 1.22.0 w

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 231.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 317.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 270.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 215.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 299.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 252.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 228.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with

Set TTS

In [ ]:
import torch
from TTS.api import TTS

#Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path_tammy="/content/tts_models/tammy09apr/"
config_path_tammy="/content/tts_models/tammy09apr/config.json"

# Init TTS
tts = TTS("tts_models/en/ljspeech/tacotron2-DDC")  # Load the desired model
ttsTammy = TTS(model_path_tammy, config_path_tammy)  # Load the desired model
# Run TTS
tts.tts_to_file(text="Hello world!", speaker=None, file_path="output.wav")
ttsT.tts_to_file(text="Hello world!", speaker=None, file_path="outputTammy.wav")


RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC


 97%|█████████▋| 109M/113M [00:03<00:00, 33.7MiB/s]

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2



100%|██████████| 113M/113M [00:04<00:00, 25.9MiB/s]


 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
Removing weight norm...


KeyError: ''